In [0]:

##### CNN Grid Search

In [0]:
import pandas as pd

In [0]:
import numpy as np

In [0]:
df = pd.read_csv("Work_with_Ambuj.csv")

In [0]:
df.set_index('Date', inplace = True)


In [0]:
df.index = pd.to_datetime(df.index)

In [0]:
df2  = df.as_matrix()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [0]:
dataset = (df2-df2.min())/(df2.max()-df2.min())


In [0]:
from numpy import array

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

Using TensorFlow backend.


In [0]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [0]:



# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
# find the end of this pattern
        end_ix = i + n_steps
# check if we are beyond the dataset
        if end_ix > len(sequences):
            break
# gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


In [0]:
n_steps = 2
# convert into input/output
X, y = split_sequences(dataset, n_steps)

In [0]:
x_train = X[:1306]
y_train = y[:1306]
x_test = X[1306:]
y_test = y[1306:]

In [0]:
n_features = x_train.shape[2]

In [0]:
# Use scikit-learn to grid search the batch size and epochs

# Function to create model, required for KerasClassifier
def create_model(neurons=1, hidden_layers = 1):
  
	# create model
  for i in range(hidden_layers):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(n_steps,n_features)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(neurons, activation='relu'))
  model.add(Dense(1))
  model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
  return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [5]
epochs = [5,10]
hidden_layers = [1,2]
neurons = [2,4]
param_grid = dict(batch_size=batch_size, epochs=epochs, hidden_layers = hidden_layers, neurons = neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(x_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Best: 0.002297 using {'batch_size': 5, 'epochs': 10, 'hidden_layers': 2, 'neurons': 2}
0.001531 (0.002167) with: {'batch_size': 5, 'epochs': 5, 'hidden_layers': 1, 'neurons': 2}
0.000000 (0.000000) with: {'batch_size': 5, 'epochs': 5, 'hidden_layers': 1, 'neurons': 4}
0.000000 (0.000000) with: {'batch_size': 5, 'epochs': 5, 'hidden_layers': 2, 'neurons': 2}
0.000000 (0.000000) with: {'batch_size': 5, 'epochs': 5, 'hidden_layers': 2, 'neurons': 4}
0.000000 (0.000000) with: {'batch_size': 5, 'epochs': 10, 'hidden_layers': 1, 'neurons': 2}
0.000000 (0.000000) with: {'batch_size': 5, 'epochs': 10, 'hidden_layers': 1, 'neurons': 4}
0.002297 (0.001877) with: {'batch_size': 5, 'epochs': 10, 'hidden_layers': 2, 'neurons': 2}
0.002297 (0.001876) with: {'batch_size': 5, 'epochs': 10, 'hidden_layers': 2, 'neurons': 4}


In [0]:
#### Multi_Headed_CNN


In [0]:
from numpy import array
from numpy import hstack
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
from sklearn.metrics import r2_score
from keras.layers import Dropout

In [0]:
def split_sequenc(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
# find the end of this pattern
        end_ix = i + n_steps
# check if we are beyond the dataset
        if end_ix > len(sequences):
            break
# gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


In [0]:
n_steps = 3
# convert into input/output
X, y = split_sequenc(dataset, n_steps)

In [0]:
print(X.shape, y.shape)
x_train = X[:1306]
y_train = y[:1306]
x_test = X[1306:]
y_test = y[1306:]
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(1563, 3, 20) (1563,)
(1306, 3, 20)
(1306,)
(257, 3, 20)
(257,)


In [0]:
n_features = 1

In [0]:

# separate input data
X1 = x_train[:, :, 0].reshape(x_train.shape[0], x_train.shape[1], n_features)
X2 = x_train[:, :, 1].reshape(x_train.shape[0], x_train.shape[1], n_features)
X3 = x_train[:, :, 2].reshape(x_train.shape[0], x_train.shape[1], n_features)
X4 = x_train[:, :, 3].reshape(x_train.shape[0], x_train.shape[1], n_features)
X5 = x_train[:, :, 4].reshape(x_train.shape[0], x_train.shape[1], n_features)
X6 = x_train[:, :, 5].reshape(x_train.shape[0], x_train.shape[1], n_features)
X7 = x_train[:, :, 6].reshape(x_train.shape[0], x_train.shape[1], n_features)

X8 = x_train[:, :, 7].reshape(x_train.shape[0], x_train.shape[1], n_features)
X9 = x_train[:, :, 8].reshape(x_train.shape[0], x_train.shape[1], n_features)
X10 = x_train[:, :, 9].reshape(x_train.shape[0], x_train.shape[1], n_features)
X11 = x_train[:, :, 10].reshape(x_train.shape[0], x_train.shape[1], n_features)
X12 = x_train[:, :, 11].reshape(x_train.shape[0], x_train.shape[1], n_features)
X13 = x_train[:, :, 12].reshape(x_train.shape[0], x_train.shape[1], n_features)
X14 = x_train[:, :, 13].reshape(x_train.shape[0], x_train.shape[1], n_features)
X15 = x_train[:, :, 14].reshape(x_train.shape[0], x_train.shape[1], n_features)
X16 = x_train[:, :, 15].reshape(x_train.shape[0], x_train.shape[1], n_features)
X17 = x_train[:, :, 16].reshape(x_train.shape[0], x_train.shape[1], n_features)
X18 = x_train[:, :, 17].reshape(x_train.shape[0], x_train.shape[1], n_features)

X19 = x_train[:, :, 18].reshape(x_train.shape[0], x_train.shape[1], n_features)
X20 = x_train[:, :, 19].reshape(x_train.shape[0], x_train.shape[1], n_features)

In [0]:
AX1 = x_test[:, :, 0].reshape(x_test.shape[0], x_test.shape[1], n_features)
AX2 = x_test[:, :, 1].reshape(x_test.shape[0], x_test.shape[1], n_features)
AX3 = x_test[:, :, 2].reshape(x_test.shape[0], x_test.shape[1], n_features)
AX4 = x_test[:, :, 3].reshape(x_test.shape[0], x_test.shape[1], n_features)
AX5 = x_test[:, :, 4].reshape(x_test.shape[0], x_test.shape[1], n_features)
AX6 = x_test[:, :, 5].reshape(x_test.shape[0], x_test.shape[1], n_features)
AX7 = x_test[:, :, 6].reshape(x_test.shape[0], x_test.shape[1], n_features)

AX8 = x_test[:, :, 7].reshape(x_test.shape[0], x_test.shape[1], n_features)
AX9 = x_test[:, :, 8].reshape(x_test.shape[0], x_test.shape[1], n_features)
AX10 = x_test[:, :, 9].reshape(x_test.shape[0], x_test.shape[1], n_features)
AX11 = x_test[:, :, 10].reshape(x_test.shape[0], x_test.shape[1], n_features)
AX12 = x_test[:, :, 11].reshape(x_test.shape[0], x_test.shape[1], n_features)
AX13 = x_test[:, :, 12].reshape(x_test.shape[0], x_test.shape[1], n_features)
AX14 = x_test[:, :, 13].reshape(x_test.shape[0], x_test.shape[1], n_features)

AX15 = x_test[:, :, 14].reshape(x_test.shape[0], x_test.shape[1], n_features)
AX16 = x_test[:, :, 15].reshape(x_test.shape[0], x_test.shape[1], n_features)
AX17 = x_test[:, :, 16].reshape(x_test.shape[0], x_test.shape[1], n_features)
AX18 = x_test[:, :, 17].reshape(x_test.shape[0], x_test.shape[1], n_features)
AX19 = x_test[:, :, 18].reshape(x_test.shape[0], x_test.shape[1], n_features)
AX20 = x_test[:, :, 19].reshape(x_test.shape[0], x_test.shape[1], n_features)




In [0]:
from keras.optimizers import Adam
#opt = SGD(lr = 0.01, momentum = 0.9, decay = 0.01)

opt = Adam(lr=0.001, beta_1=0.99, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [0]:
def create_model(neurons=1, hidden_layers = 1):
  
	# create model
  for i in range(hidden_layers):
    
    visible1 = Input(shape=(n_steps, n_features))
    cnn1 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible1)
    cnn1 = MaxPooling1D(pool_size=2)(cnn1)
    cnn1 = Flatten()(cnn1)

    visible2 = Input(shape=(n_steps, n_features))
    cnn2 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible2)
    cnn2 = MaxPooling1D(pool_size=2)(cnn2)
    cnn2 = Flatten()(cnn2)

    visible3 = Input(shape=(n_steps, n_features))
    cnn3 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible3)
    cnn3 = MaxPooling1D(pool_size=2)(cnn3)
    cnn3 = Flatten()(cnn3)

    visible4 = Input(shape=(n_steps, n_features))
    cnn4 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible4)
    cnn4 = MaxPooling1D(pool_size=2)(cnn4)
    cnn4 = Flatten()(cnn4)

    visible5 = Input(shape=(n_steps, n_features))
    cnn5 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible5)
    cnn5 = MaxPooling1D(pool_size=2)(cnn5)
    cnn5 = Flatten()(cnn5)

    visible6 = Input(shape=(n_steps, n_features))
    cnn6 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible6)
    cnn6 = MaxPooling1D(pool_size=2)(cnn6)
    cnn6 = Flatten()(cnn6)

    visible7 = Input(shape=(n_steps, n_features))
    cnn7 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible7)
    cnn7 = MaxPooling1D(pool_size=2)(cnn7)
    cnn7 = Flatten()(cnn7)

    visible8 = Input(shape=(n_steps, n_features))
    cnn8 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible8)
    cnn8 = MaxPooling1D(pool_size=2)(cnn8)
    cnn8 = Flatten()(cnn8)

    visible9 = Input(shape=(n_steps, n_features))
    cnn9 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible9)
    cnn9 = MaxPooling1D(pool_size=2)(cnn9)
    cnn9 = Flatten()(cnn9)

    visible10 = Input(shape=(n_steps, n_features))
    cnn10 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible10)
    cnn10 = MaxPooling1D(pool_size=2)(cnn10)
    cnn10 = Flatten()(cnn10)

    visible11 = Input(shape=(n_steps, n_features))
    cnn11 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible11)
    cnn11 = MaxPooling1D(pool_size=2)(cnn11)
    cnn11 = Flatten()(cnn11)

    visible12 = Input(shape=(n_steps, n_features))
    cnn12 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible12)
    cnn12 = MaxPooling1D(pool_size=2)(cnn12)
    cnn12 = Flatten()(cnn12)

    visible13 = Input(shape=(n_steps, n_features))
    cnn13 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible13)
    cnn13 = MaxPooling1D(pool_size=2)(cnn13)
    cnn13 = Flatten()(cnn13)

    visible14 = Input(shape=(n_steps, n_features))
    cnn14 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible14)
    cnn14 = MaxPooling1D(pool_size=2)(cnn14)
    cnn14= Flatten()(cnn14)

    visible15 = Input(shape=(n_steps, n_features))
    cnn15 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible15)
    cnn15 = MaxPooling1D(pool_size=2)(cnn15)
    cnn15 = Flatten()(cnn15)

    visible16 = Input(shape=(n_steps, n_features))
    cnn16 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible16)
    cnn16 = MaxPooling1D(pool_size=2)(cnn16)
    cnn16 = Flatten()(cnn16)

    visible17 = Input(shape=(n_steps, n_features))
    cnn17 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible17)
    cnn17 = MaxPooling1D(pool_size=2)(cnn17)
    cnn17 = Flatten()(cnn17)

    visible18 = Input(shape=(n_steps, n_features))
    cnn18 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible18)
    cnn18 = MaxPooling1D(pool_size=2)(cnn18)
    cnn18 = Flatten()(cnn18)

    visible19 = Input(shape=(n_steps, n_features))
    cnn19 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible19)
    cnn19 = MaxPooling1D(pool_size=2)(cnn19)
    cnn19 = Flatten()(cnn19)

    visible20 = Input(shape=(n_steps, n_features))
    cnn20 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible20)
    cnn20 = MaxPooling1D(pool_size=2)(cnn20)
    cnn20 = Flatten()(cnn20)

    merge = concatenate([cnn1, cnn2, cnn3, cnn4, cnn5, cnn6, cnn7, cnn8, cnn9, cnn10, cnn11, cnn12,cnn13, cnn14, cnn15, cnn16,cnn17, cnn18, cnn19, cnn20])

    merge = concatenate([cnn1, cnn2, cnn3, cnn4, cnn5, cnn6, cnn7, cnn8, cnn9, cnn10, cnn11, cnn12,cnn13, cnn14, cnn15, cnn16,cnn17, cnn18, cnn19, cnn20])
    dense = Dense(neurons, activation='relu')(merge)
    #dense = Dropout(0.2)(densea)
    output = Dense(1)(dense)
    model = Model(inputs=[visible1, visible2, visible3, visible4, visible5, visible6, visible7, visible8, visible9, visible10, visible11, visible12, visible13, visible14, visible15, visible16,visible17, visible18, visible19, visible20], outputs=output)
  model.add(Dense(1))
  model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
  return model

In [0]:
x_t = [X1, X2, X3, X4, X5, X6,X7, X8, X9, X10, X11, X12, X13,X14,X15, X16, X17, X18, X19, X20]
t_x_t = [AX1, AX2, AX3, AX4, AX5, AX6,AX7, AX8, AX9, AX10, AX11, AX12, AX13,AX14,AX15, AX16, AX17, AX18, AX19, AX20]

In [0]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [5]
epochs = [5,10]
hidden_layers = [1,2]
neurons = [2,4]
param_grid = dict(batch_size=batch_size, epochs=epochs, hidden_layers = hidden_layers, neurons = neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(x_t, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


ValueError: ignored